In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import string
import re
pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)

<Figure size 432x288 with 0 Axes>

In [2]:
sharks = pd.read_excel('C:/Users/klari/Documents/Learning/a Datasets/GSAF5 (sharks).xls', sort =False)

In [3]:
sharks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Sex_clean,Unnamed: 23
0,2020.05.09,09-May-2020,2020.0,Unprovoked,USA,California,"Sand Dollar Beach, Santa Cruz County",Surfing,Ben Kelly,M,...,White shark,"R. Collier, GSAF, K. McMurray, TrackingSharks.com",2020.05.09-Kelly.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.05.09,2020.05.09,6523.0,NaN,NaN
1,2020.05.08,08-May-2020,2020.0,Unprovoked,AUSTRALIA,Victoria,"Southside Beach, near Geelong",Surfing,Dylan Nacass,M,...,NaN,"K. McMurray, TrackingSharks.com",2020.05.08-Nacass.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.05.08,2020.05.08,6522.0,NaN,NaN
2,2020.05.01,01-May-2020,2020.0,Unprovoked,USA,California,"Summerland, Santa Barbara County",Swimming,Mandy Boyd,F,...,5' to 6' shark,"R. Collier, GSAF, K. McMurray, TrackingSharks.com",2020.05.01-Boyd.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.05.01,2020.05.01,6521.0,NaN,NaN
3,2020.04.29,29-Apr-2020,2020.0,NaN,USA,California,"Moonlight Beach, San Diego County",Body Boarding,male,M,...,NaN,"K. McMurray, TrackingSharks.com and M. Michael...",2020.04.29-Encinitas.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.04.29,2020.04.29,6520.0,NaN,NaN
4,2020.04.21,21-Apr-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,Killick Creek near Crescent Head,Swimming,female,F,...,NaN,"B. Myatt, GSAF & K. McMurray, TrackingSharks.com",2020.04.21-CrescentHead.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.04.21,2020.04.21,6519.0,NaN,NaN


In [4]:
sharks.dropna(subset = ["Date"], inplace=True)

In [5]:
sharks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Sex_clean',
       'Unnamed: 23'],
      dtype='object')

In [6]:
sharks = sharks.drop(columns = [ 'Unnamed: 23', 'href formula'])

## Clean 'Country'

In [7]:
#set(sharks.Country)

In [8]:
sharks['Country2'] = [str(x).strip().upper() for x in sharks['Country']]    
sharks['Country'] = sharks['Country2']
sharks = sharks.drop(columns = ['Country2'])

In [9]:
#sharks[sharks['Country'].str.contains(r'.+\?')]
sharks['Country'] = sharks['Country'].str.replace('?', '')

In [10]:
sharks.loc[sharks['Country'] == 'COAST OF AFRICA', 'Country'] = 'AFRICA'
sharks.loc[sharks['Country'] == 'CEYLON (SRI LANKA)', 'Country'] = 'CEYLON'
sharks.loc[sharks['Country'] == 'REUNION', 'Country'] = 'REUNION ISLAND'
sharks.loc[sharks['Country'] == 'MALDIVES', 'Country'] = 'MALDIVE ISLANDS'
sharks.loc[sharks['Country'] == 'UNITED ARAB EMIRATES (UAE)', 'Country'] = 'UNITED ARAB EMIRATES'
sharks.loc[sharks['Country'] == 'ST HELENA, British overseas territory', 'Country'] = 'ST. HELENA'
sharks.loc[sharks['Country'] == 'COLUMBIA', 'Country'] = 'COLOMBIA'
sharks.loc[sharks['Country'] == 'SOUTHWEST PACIFIC OCEAN', 'Country'] = 'SOUTH PACIFIC OCEAN'
sharks.loc[sharks['Country'] == 'FEDERATED STATES OF MICRONESIA', 'Country'] = 'MICRONESIA'
sharks.loc[sharks['Country'] == 'TOBAGO', 'Country'] = 'TRINIDAD & TOBAGO'
sharks.loc[sharks['Country'] == 'WESTERN SAMOA', 'Country'] = 'SAMOA'
sharks.loc[sharks['Country'] == 'MID-PACIFC OCEAN', 'Country'] = 'MID PACIFIC OCEAN' 
sharks.loc[sharks['Country'] == 'SOLOMON ISLANDS', 'Country'] = 'SOLOMON ISLANDS / VANUATU'
sharks.loc[sharks['Country'] == 'VANUATU', 'Country'] = 'SOLOMON ISLANDS / VANUATU'
sharks.loc[sharks['Country'] == 'RED SEA / INDIAN OCEAN', 'Country'] = 'RED SEA'

## Clean 'Year'

In [11]:
sharks = sharks[sharks['Year'].between(1, 1000, inclusive=True) == False]

In [12]:
sharks = sharks[sharks['Date'].str.contains(r'.+B\.C\.') == False]

In [13]:
sharks['Year2'] = sharks['Date'].str.extract(r'([0-9]{4})')
sharks.loc[sharks['Year']==0, 'Year'] = sharks['Year2']

#some of the years extracted above are approximated. ie, 'Before 1905' is too vague to estimate; I've taken 1905 as the year. This could vary in degree of incorrectness- actual may be 1904 or 1704. 
#possibly should be dropped altogether- will decide when histogram of year is viewed

sharks.loc[sharks['Year'].isna() == True, 'Year'] = sharks['Year2']
sharks = sharks.drop(columns = ['Year2'])
sharks.loc[sharks['Date'] == 'World War II', 'Year'] = '1940'
sharks['Year'] = sharks['Year'].replace("'", '')
sharks = sharks[sharks['Year'].isna() == False]
sharks['Year'] = sharks['Year'].astype('int')

In [14]:
#sharks['Year'].unique()

## Clean 'Type'

In [15]:
sharks['Type'].unique()

array(['Unprovoked', nan, 'Provoked', 'Questionable', 'Watercraft',
       'Unverified', 'Invalid', 'Under investigation', 'Sea Disaster',
       'Unconfirmed', 'Boat'], dtype=object)

In [16]:
sharks = sharks[sharks['Case Number'] != '2020.04.29']
sharks['Type'][sharks['Type'].isna() == True] = 'Unprovoked'

In [17]:
sharks = sharks[sharks['Type'] != 'Invalid'] #if invalid, will only confuse data
sharks.loc[sharks['Type'] == 'Boat', 'Type'] = 'Watercraft' #was going to remove these, but will actually switch to 'Watercraft' for consistency
sharks = sharks[sharks['Type'] != 'Under investigation'] # there is only one so just going to drop it
sharks = sharks[sharks['Type'] != 'Unconfirmed'] # there is only one so just going to drop it- missing much info so don't want to coerce into unprovoked
sharks.loc[(sharks['Type'].isna() == True) & (sharks['Activity'] == 'Wreck of a sampam'), 'Type'] = 'Sea Disaster'
sharks.loc[sharks['Type'] == 'Unverified', 'Type'] = 'Unprovoked' #only 1; acting as if legit

## Clean 'Activity'

In [18]:
#set(sharks['Activity'])

In [19]:
#set(sharks['Activity'][(sharks['Activity'].str.contains(r'(swim|stand|wading|Swim|Stand|Wading)')) & (~sharks['Activity'].str.contains(r'(Fish|fish|apsize|shark|Shark|porpoise|oyster|whale|dolphin|perlemoen|abalone|slaughterhouse)', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False))])
sharks['Activity_collapsed'] = ''
# includes: (Attempt|Attack|harpooned|dead|Catching|Chumming|captured|force-feed|Kiss|feeding|photographing|Dragging|filming|Feeding|Feeling|Filming|Finning|Fell|Fishing|Gaffing|Grab|Hand feed|Harass|Harpoon|Hauling|Help|Hoist|Hold|Hunt|Inspect|Investigat|landed on|Killing|fishing|Measuring|Moving|Netting|Observing|Petting|Photographing|Picking|Pulling|foot|hand|Removing|Restrain|Reviv|catch|Shooting|Grab|Slap|lasso|pulled|shot|Spearing|Standing|stepped|Stuffing|feeding|Swimming with|Tagging|Teasing|Testing|Thrashing|Touching|netted|Watching|Wrangling|)

sharks.loc[(sharks['Activity'].str.contains(r'(collided|[C|c]apsize|aground|[S|s]hipwreck|210-ton brig|[W|w]reck|[D|d]isaster|[A|a]ccident|founder|crash|overturn|stove|sank)', na=False)) | (sharks['Type'].str.contains('Sea Disaster', na=False)), 'Activity_collapsed'] = 'Sea disaster/capsize/wreck/accident'
sharks.loc[(sharks['Activity'].str.contains(r'( fish|[F|f]ish|[C|c]lam|porpoise|whale|dolphin|perlemoen|[A|a]balone|crayfish|slaughterhouse|bait|[T|t]rochus|opihi|squid|shad|bluefish|net|[C|c]rab|conch|beche-de-mer|trepang|tuna|shellfish|[L|l]obster|[S|s]hrimp|[P|p]rawn|meat|turtle|seal|[O|o]yster|[A|a]ngling|sardine|bichiques|[C|c]hum|[S|s]almon|[F|f]lounder|sea urchins|sponges)', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False)) & (~sharks['Activity'].str.contains(r'Hawaiian brig|Observing', na=False)), 'Activity_collapsed'] = 'Fishing / Proximity to Sea Life'
sharks.loc[(sharks['Activity'].str.contains(r'(SUP|[S|s]wim|[S|s]ta[m|n]d|[W|w]ad[e|ing]|[B|b]athing|[D|d]iv[e|ing]|[S|s]urf|[W|w]alk|[S|s]it|[B|b]oard|Playing|Snorkeling|Floating|[F|f]eet|Cling|Clean|Lying|Adrift|Kneeling|Holding|Paddling|Splash|Wash|[F|f]ell|Crouch|banana|Filming$|documentary|Jump|Rest|Sitting|Tread|[S|s]ki)')) & (~sharks['Activity'].str.contains(r'(Chasing|[C|c]apsize|aground|[S|s]hipwreck|210-ton brig|[W|w]reck|[D|d]isaster|[A|a]ccident|feeding| fish|[F|f]ish|[C|c]lam|porpoise|oyster|whale|dolphin|perlemoen|[A|a]balone|crayfish|slaughterhouse|bait|trochus|opihi|squid|bluefish|[C|c]rab|conch|beche-de-mer|trepang|tuna|shellfish|[L|l]obster|[S|s]hrimp|meat|turtle|seal|[O|o]yster|[C|c]hum|[S|s]almon|sea urchins|sponges|Feeding|holding shark|putting hand|shark tank|stepped on|grabbed shark)', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False)), 'Activity_collapsed'] = 'Unprovoked' #'Swimming/Wading/Standing/Diving etc'
sharks.loc[(sharks['Activity'].str.contains(r'(shark|Shark)', na=False)) & (~sharks['Activity'].str.contains(r'surfmat|bluefish|squid|Fukulya Maru|bathing|capsize|Crossing inlet|trochus|wife|tarpon|dory|Len Bedford|grouper|opihi|menaced|warn bathers|possibly ascended into path', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False)), 'Activity_collapsed'] = 'Provoked'
sharks.loc[sharks['Activity'] == 'Chasing shark out of bathing area while riding on a surf-ski', 'Activity_collapsed'] = 'Provoked'



sharks.loc[(sharks['Activity'].str.contains('[C|c]anoeing', na=False)) | (sharks['Activity'] == 'Kayaking') | (sharks['Activity'] == 'Kakaying') | (sharks['Activity'] == 'Kayaking ') | (sharks['Activity'] == 'Boat') | (sharks['Activity'] == 'Rowing') | (sharks['Activity'] == 'Sculling') | (sharks['Activity'] == 'Sailing') | (sharks['Activity'] == 'Watercraft') | (sharks['Activity'] == 'Transatlantic Rowing') | (sharks['Activity'] == 'Cruising') | (sharks['Activity'] == 'Rowing') | (sharks['Activity'] == 'Yacht race') | (sharks['Activity'].str.contains('Rowing', na=False)) | (sharks['Name'].str.contains('dinghy|[B|b]oat|ferry|yacht|skiff|cutter')), 'Activity_collapsed'] = 'Watercraft'

C:\Users\klari\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [20]:
set(sharks['Activity'][sharks['Activity_collapsed'] == ''])
#sharks.groupby(['Activity', 'Activity_collapsed']).size()

{'   ',
 ' a canoe was pursuing a schooner that had forcibily abducted 5 young girls',
 '"Climbing up to ship after repairing the stern in water"',
 '"Crossing the river"',
 '3 men & 2 boys picked up wearing life jackets and with inner tube',
 'Attempting to fix motor',
 'Attempting to rescue drowning man',
 'Attempting to rescue shipmate',
 'Attempting to retreive a dinghy',
 'Attempting to set underwater endurance record',
 'Batin',
 'Bending over',
 'Boat stopped to repair electric pump',
 'Boat swamped',
 'Boating',
 'Body found on deserted luxury yacht, 38’ Christine',
 'Canoe swamped',
 'Climbing back on ship',
 'Collecting aquarium specimens',
 'Collecting marine specimens',
 'Collecting shells',
 'Coming ashore on a hawser',
 'Conducting research',
 'Crossing inlet in a boat, seen fighting sharks with his oar, sharks smashed boat',
 'Crossing river on a raft',
 'Crossing the bay at the ford',
 'Crossing the river mouth',
 "Days before the surrender of Singapore, the 3 men escap

In [21]:
sharks['Activity_collapsed'].unique()
#set(sharks['Activity'][sharks['Activity_collapsed'] == 'Fishing / Proximity to Sea Life'])
#set(sharks['Activity'][sharks['Activity_collapsed'] == 'Provoked'])
#set(sharks['Activity'][sharks['Activity_collapsed'] == 'Sea disaster/capsize/wreck/accident'])
#set(sharks['Activity'][sharks['Activity_collapsed'] == 'Unprovoked']) 
#set(sharks['Activity'][sharks['Activity_collapsed'] == 'Watercraft'])

array(['Unprovoked', '', 'Fishing / Proximity to Sea Life', 'Watercraft',
       'Provoked', 'Sea disaster/capsize/wreck/accident'], dtype=object)

In [22]:
sharks[['Activity', 'Type']][(sharks['Activity_collapsed'] == '') & (sharks['Activity'].isna() == False)] 

#OKAY I am real tired of cleaning this data.... 
sharks.loc[(sharks['Activity_collapsed'] == '') & (sharks['Activity'].isna() == False), 'Activity_collapsed'] = sharks.loc[(sharks['Activity_collapsed'] == '') & (sharks['Activity'].isna() == False), 'Type']

In [23]:
#fill nans
#just going to have to take their work for it.....tempted to drop due to lack of info, but it's not a huge dataset. Then again, that means if they are wrong they will have a larger impact
len(sharks[['Activity','Type']][(sharks['Activity_collapsed'] == '') & (sharks['Activity'].isna() == True)])

sharks.loc[(sharks['Activity_collapsed'] == '') & (sharks['Activity'].isna() == True), 'Activity_collapsed'] = sharks.loc[(sharks['Activity_collapsed'] == '') & (sharks['Activity'].isna() == True), 'Type'] = sharks['Type'].astype(str) + '_TEMP'#'TempAssign'


In [24]:
#Will explore data further then decide whether to keep.
sharks[['Activity_collapsed', 'Activity']][sharks['Type'] == 'Provoked']
#sharks[['Activity_collapsed', 'Activity']][sharks['Type'] == 'Questionable']
#sharks[['Activity_collapsed', 'Activity']][sharks['Type'] == 'Unprovoked']
#sharks[['Activity_collapsed', 'Activity']][sharks['Type'] == 'Watercraft']

set(sharks['Activity_collapsed'])

{'Fishing / Proximity to Sea Life',
 'Provoked',
 'Provoked_TEMP',
 'Questionable_TEMP',
 'Sea disaster/capsize/wreck/accident',
 'Unprovoked',
 'Unprovoked_TEMP',
 'Watercraft',
 'Watercraft_TEMP'}

## Clean 'Sex'

In [25]:
#set(sharks['Area'])
#set(sharks['Location'])
set(sharks['Sex '])

{'.', 'F', 'M', 'M ', 'M x 2', 'N', 'lli', nan}

In [26]:
# ORIGINAL

sharks.loc[sharks['Sex '] == 'F', 'Sex_clean'] = 'F'
sharks.loc[sharks['Sex '] == 'M', 'Sex_clean'] = 'M'

# FEMALE
sharks.loc[sharks['Name'].str.contains('female', na=False), 'Sex_clean'] = 'F'

# MALE
sharks.loc[(sharks['Sex '] == 'M ') | (sharks['Sex '] == 'lli') | (sharks['Name'].str.contains('(?<![F|f]e)male|schoolboy')), 'Sex_clean'] = 'M'
sharks.loc[sharks['Name'].str.contains(r'^Frederic|^Jose|^Laurent|^Dr. Leo|^Sergio|^Dr. George|^Ian|^Tony|^Vittorio|^Captain Jack|^Maurice|^William|^John|^Emile|^Jeff|^Miller|^male|^Gilvan|^Shawn|^Stephen|^Yuji|^Josebias|^Ben|^Jurandir|^Nagisa|^Yuji', na=False), 'Sex_clean'] = 'M'

# UNKNOWN
sharks.loc[(sharks['Name'].isna()) & (sharks['Sex '].isna()), 'Sex_clean'] = 'U' 
sharks.loc[(sharks['Name'].str.contains(r'^[A-Z]\.')) | (sharks['Sex '] == '\.') | (sharks['Name'].str.contains('child|teen|Unknown|Unidentified|Anonymous|student|teacher|aquarist|swimmer|anonymous|a native|a resident|a surfer|U.S. citizen|Fimler|^a youth', na=False)), 'Sex_clean'] = 'U'

# MULTI
sharks.loc[(sharks['Name'].str.contains(r'[O|o]ccupants|[P|p]assengers|[M|m]igrants|[R|r]efugees|[P|p]oachers|[P|p]eople|[R|r]owers|[C|c]rew|pilgims|Fijians|slaves|youths|passenger|tourists|fishermen|friends|swimmers|men|pilgrims|bathers|\&|Swimming', na=False)) | (sharks['Sex '] == 'M x 2'), 'Sex_clean'] = 'Multi'
sharks.loc[sharks['Case Number'].str.contains('1934.07.11|1959.07.03.a & b', na=False), 'Sex_clean'] = 'Multi' # M&F

#SHIPS/NA
sharks.loc[(sharks['Activity_collapsed'] == 'Watercraft') | (sharks['Name'].str.contains(r'dinghy|[B|b]oat|ferry|yacht|skiff|cutter|paddle|launch|Hubeh|Leonida|trawler') & (~sharks['Name'].str.contains(r'[C|c]rew', na=False))), 'Sex_clean'] = 'N/A'

In [93]:
print(len(sharks[sharks['Sex_clean'].isna() == True]))
set(sharks['Sex_clean'])

#sharks[sharks['Sex '].isna() == True] 

25


{'F', 'M', 'Multi', 'N/A', 'U', nan}

In [28]:
sharks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href', 'Case Number.1',
       'Case Number.2', 'original order', 'Sex_clean', 'Activity_collapsed'],
      dtype='object')

## Clean 'Species'

In [30]:
sharks['Species'] = ['' for x in np.arange(len(sharks['Species ']))]
sharks = sharks.reset_index()
sharks['Species '] = sharks['Species '].fillna('NA')

In [31]:
for x in np.arange(len(sharks)):
    if re.search('[S|s]even[\s|\-]?gill|7[\s|\-]?gill' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species'] = 'Seven-gill shark'
    elif re.search('[T|t]hresher' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species'] = 'Thresher shark'
    elif re.search('[W|w]obbegong' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species'] = 'Wobbegong shark'
    elif re.search('[L|l]emon' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species'] =  'Lemon shark'
    elif re.search('[L|l]eopard' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species'] =  'Leopard shark'
    elif re.search('[H|h]ammerhead' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species'] =  'Hammerhead shark'
    elif re.search('(?<![S|s]and )[T|t]iger' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species'] = 'Tiger shark'
    elif re.search('(?<!([G|g]r[e|a]y|[B|b]lue)[\s|\-])[N|n]urse' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Nurse shark'
    elif re.search('[B|b]lack[\s|-]?tip' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Blacktip shark'
    elif re.search('[S|s]pinner' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Spinner shark'
    elif re.search('[D|d]usky' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Dusky shark'
    elif re.search('[B|b]ull|[Z|z]ambe[z|s]i|C. leucas|Lake Nicaragua' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Bull shark'
    elif re.search('[B|b]ronze|(?<![W|w]hite[\s|\-]tipped\s])[W|w]haler|[C|c]opper|[N|n]arrow[\s|\-]?tooth' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Copper shark'
    elif re.search('[B|b]lack[\s|-]?fin' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Blackfin shark'
    elif re.search('[B|b]lue[\s|-]?nose|blunt[\s|\-]?nose|six[\s|\-]?gill|6[\s|\-]?gill' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] =  'Bluntnose sixgill shark'
    elif re.search('(?<!([[L|l]esser\s))[W|w]hite |[W|w]hite pointer' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Great white shark'
    elif re.search('[M|m]ako|[B|b]lue pointer|[B|b]onit[a|o]' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Mako shark'
    elif re.search('[P|p]orbeagle' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Porbeagle shark'
    elif re.search('(?<=([G|g]r[e|a]y|[B|b]lue)[\s|\-])[N|n]urse|[W|w]hite[\s|\-]?tip|[N|n]igano|[S|s]ilver[\s|\-]?tip|brown|(?<![B|b]ronze\s)[W|w]haler|(?<![S|s]potted )[R|r]agged[\s|\-]?tooth|[S|s]and[\s]?bar' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Oceanic whitetip shark'
    elif re.search('[D|d]og|mud|piked|dogfish' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Spiny dogfish'
    elif re.search('(?<![G|g]r[e|a]y\s)[R|r]eef' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Caribbean reef shark'
    elif re.search('[G|g]r[e|a]y\s(?!nurse)(?!whaler)' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Grey reef shark'
    elif re.search('C\.\salbimarginatus|[S|s]ilver[\s|\-]?tip' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Silvertip shark'
    elif re.search('[S|s]ilk|blackspot|[G|g]r[a|e]y\s(?=whaler)|olive|[R|r]idgeback|[S|s]ickle' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Silky shark'
    elif re.search('[G|g]ummy' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Gummy shark'
    elif re.search('[C|c]ow' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Cow shark'
    elif re.search('[C|c]arpet' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Carpet shark'
    elif re.search('[B|b]lue(?![\-|\s]{0,1}tip)(?!\s{1}pointer)(?![\s|\-]{1}nurse)' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Blue shark'
    elif re.search('(?<=[B|b]lue )[N|n]urse|(?<=[G|g]r[e|a]y )[N|n]urse|[S|s]and(?!bar)|(?<=[S|s]potted )[R|r]agged[\s|\-]?tooth' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Sand shark'
    elif re.search('[B|b]anjo' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Banjo shark - not a real shark'
    elif re.search('[S|s]hovelnose' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Shovelnose shark - not a real shark'
    elif re.search('[S|s]mooth[\s|\-]?hound' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Smooth-hound shark'
    elif re.search('[G|g]oblin' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Goblin shark'
    elif re.search('[C|c]ookie[\s|\-]?cutter|[C|c]igar' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Cookiecutter shark'
    elif re.search('[G|g]alapagos' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Galapagos shark'
    elif re.search('[C|c]atshark' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Catshark'
    elif re.search('[P|p]ort [J|j]ackson' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Port Jackson shark'
    elif re.search('[A|a]ngel|[M|m]onkfish' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Angelshark'
    elif re.search('[W|w]hale(?<!er)' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'Whale shark'
    elif re.search('[S|s]oupfin|[S|s]chool|[S|s]napper|[T|t]ope' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species'] = 'School shark'
#    elif re.search('' , sharks.loc[x, 'Species ']): 
#        sharks.loc[x,  'Species'] = ''
    elif sharks.loc[x, 'Species '] == 'NA':
         sharks.loc[x,  'Species'] = 'No details'
    else:
        ''

C:\Users\klari\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Possible nested set at position 6


In [32]:
# none are missing :) although some are overlapping (I know some had multiple types of sharks in the string- something to be looked at)
set(sharks['Species'])

{'',
 'Angelshark',
 'Banjo shark - not a real shark',
 'Blackfin shark',
 'Blacktip shark',
 'Blue shark',
 'Bluntnose sixgill shark',
 'Bull shark',
 'Caribbean reef shark',
 'Carpet shark',
 'Catshark',
 'Cookiecutter shark',
 'Copper shark',
 'Cow shark',
 'Dusky shark',
 'Galapagos shark',
 'Goblin shark',
 'Great white shark',
 'Grey reef shark',
 'Gummy shark',
 'Hammerhead shark',
 'Lemon shark',
 'Leopard shark',
 'Mako shark',
 'No details',
 'Nurse shark',
 'Oceanic whitetip shark',
 'Porbeagle shark',
 'Port Jackson shark',
 'Sand shark',
 'School shark',
 'Seven-gill shark',
 'Shovelnose shark - not a real shark',
 'Silky shark',
 'Silvertip shark',
 'Smooth-hound shark',
 'Spinner shark',
 'Spiny dogfish',
 'Thresher shark',
 'Tiger shark',
 'Whale shark',
 'Wobbegong shark'}

In [64]:
#sharks['Species '] = sharks['Species '].str.replace('\\','', regex=True)

In [133]:
#set(sharks['Species '][(sharks['Species'] == 'No details') | (sharks['Species'] == '')].to_list())

In [35]:
# Note to my Canadian-metric-self:

# ' = feet 
# " = inches

In [36]:
sharks['Size_min'] =  ['' for x in np.arange(len(sharks['Species ']))]
sharks['Size_max'] =  ['' for x in np.arange(len(sharks['Species ']))]

In [263]:
sharks[['Species ','Size_min','Size_max']].head(1)

,Species,Size_min,Size_max
0,White shark,,NaN


In [260]:
for x in np.arange(len(sharks)):
    #no metres, no inches, has feet
    if re.search('[0-9]+\.?[0-9]?\s?m' , sharks.loc[x, 'Species ']) is None and re.search('[0-9]\.?[0-9]?\'{2}|\"{1}' , sharks.loc[x, 'Species ']) is None and bool(re.search('[0-9]\.?[0-9]?\'{1}(?!\')' , sharks.loc[x, 'Species '])):
        if len(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])) == 1:
            sharks.loc[x,  'Size_max'] = float(re.search('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species ']).group(1)) #* 0.3028
        elif len(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])) == 2:
            sharks.loc[x,  'Size_min'] = float(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])[0]) #* 0.3028
            sharks.loc[x,  'Size_max'] = float(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])[1]) #* 0.3028
        else:
            sharks.loc[x,  'Size_max'] = np.nan
    #has metres
    elif bool(re.search('[0-9]+\.?[0-9]?\s?m' , sharks.loc[x, 'Species '])) and (re.findall('([0-9]+\.?[0-9]?)\s?(?=[\-|to])' , sharks.loc[x, 'Species '])) is None:
        if len(re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])) == 1:
            sharks.loc[x,  'Size_max'] = re.search('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species ']).group(1)
        elif len(re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])) == 2:
            sharks.loc[x,  'Size_min'] = re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])[0]
            sharks.loc[x,  'Size_max'] = re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])[1]
        else:
             sharks.loc[x,  'Size_max'] = np.nan
        #first match having no m suffix:
    elif len(re.findall('[0-9]+\.?[0-9]*\s?m' , sharks.loc[x, 'Species '])) == 1 &  len(re.findall('([0-9]+\.?[0-9]?)\s?(?=[\-|to])' , sharks.loc[x, 'Species '])) == 1:
            sharks.loc[x,  'Size_min'] = re.findall('([0-9]+\.?[0-9]?)\s?(?=[\-|to])' , sharks.loc[x, 'Species '])[0]
            sharks.loc[x,  'Size_max'] = re.search('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species ']).group(1)
    else:
        sharks.loc[x,  'Size_max'] = np.nan

In [257]:
sharks.loc[sharks['Species '] == 'White shark 25 to 3 m', 'Size_max'] = 30 